<span style="font-size:250%">But actually, we want to analyze repos ...</span>

This notebook was aimed at developing the methodology to analyze repositories fast. Most of the early development history has been put into the [RepoAnalysis History](./RepoAnalysis_Historical.ipynb) notebook. This notebook mainly includes the actual three big analysis runs and the improvements to the analysis methods. From the results of the two notebooks, the [repoAnalysis.py](./repoAnalysis.py) module was created.

Additionally the [RepoLibrarian](./RepoLibrarian.ipynb) notebook that aims at managing downloaded repos was split off this notebook.

In [2]:
import time
from datetime import datetime
from multiprocessing import Pool
from multiprocessing import Semaphore
import multiprocessing
import sqlalchemy
import pandas
from IPython.utils import io
from IPython.display import Audio

In [3]:
%load_ext autoreload
%autoreload 2
%aimport repoAnalysis
%aimport repoLibrarian
%aimport dbUtils

---
## Preparation: Set repos work folder
(Initially I downloaded the repositories to a more local folder, which I later used a small test set for developing analysis routines)

Local folder:

In [31]:
repoLibrarian.setReposFolder('./repos/')

'./repos/'

Shared folder:

In [13]:
repoLibrarian.setReposFolder('/mnt/brick/crm20/repos/')

'/mnt/brick/crm20/repos/'

In [14]:
repoLibrarian.getReposFolder()

'/mnt/brick/crm20/repos/'

## Recap: Current state of analysis

At this point in time, the [repoAnalysis](./repoAnalysis.py) module already included the `calculateMetrics` method to calculate a given set of metrics for all files of a repository. A set of metrics has been developed (`repoAnalysis.metricSuite`). The result data should be written to database, which is why the analysis for each repository returns a pandas dataframe which can easily be written to table.

# Prerequisites to run suites and generate database

Metric suites could already be run on single repos. Now means had to be developed to do this for all repos and write the data into a given table. Note that column names have later been renamed to use snake case, because there were some errors with camel case.

In [17]:
suite = repoAnalysis.metricSuite
data = repoAnalysis.calculateMetrics(('bptlab', 'scylla', 123456789), suite)
data

Time used for ('bptlab', 'scylla', 123456789): 65.44256091117859


,sha,parent,timestamp,repo_id,loc,cloc,file_count,num_methods,num_lambdas,num_comment_lines,num_reflection,num_snakes,total_indent
0,37bcca3d0bc7b1f03818ecf24b15951b444b4f76,71e5cb71cd634f5170a4285caf2636308d5eb999,1590332677,123456789,29844,25672,276,1627,226,4235,93,1741,57175.75
1,71e5cb71cd634f5170a4285caf2636308d5eb999,a49e5be01918f8012875befe18821676dc3ce98d,1590329536,123456789,29859,25687,276,1627,226,4233,93,1741,57268.75
2,a49e5be01918f8012875befe18821676dc3ce98d,54ac50b59b456cdc156b4aa18c7be17db1511950,1588277133,123456789,29849,25675,276,1627,226,4233,93,1741,57249.75
3,54ac50b59b456cdc156b4aa18c7be17db1511950,0c42536c5657adf2e910833cc72fea577d8ee1eb,1588096131,123456789,29825,25648,276,1627,226,4230,93,1741,57069.75
4,0c42536c5657adf2e910833cc72fea577d8ee1eb,81171bf7a031077d010518344a28b0d89c6f1e90,1588095924,123456789,29781,25610,276,1627,226,4223,93,1741,56867.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,411d51771ee6c714db13c23e8912086840b84f51,9e8faf6f0b516334451c53960dee001a6e9e42c4,1485386460,123456789,12972,10850,158,691,15,1454,31,117,25773.50
476,e36dd96a1201ad35578a5e5d65a5db02135f4614,0d893a471f932b921473f72563a540da46f15fa5,1484674213,123456789,12966,10836,155,687,15,1463,30,117,25851.00
477,0d893a471f932b921473f72563a540da46f15fa5,9e8faf6f0b516334451c53960dee001a6e9e42c4,1484673978,123456789,12966,10836,155,687,15,1463,30,117,25851.00
478,9e8faf6f0b516334451c53960dee001a6e9e42c4,e1ca54916ff570656eb1d40c13b7e58c10d8bc0a,1484305822,123456789,12922,10795,155,686,15,1466,30,117,25716.50


## Exploring database creation 

In [18]:
from sqlalchemy import MetaData, Table, Column, Integer, String

Sqlalchemy provides direct methods to created tables. As columns, the metric function names can be used. Attention, the schema name has to be set!

In [22]:
columns = [Column('sha', String), Column('parent', String), Column('timestamp', Integer), Column('repo_id', Integer)]
columns = columns + list(map(lambda func: Column(func.__name__, Integer), suite))
meta = MetaData(schema='crm20')
tableName = 'lb_test2'
table = Table(
    tableName, meta,
    *columns
)
meta.create_all(dbUtils.engine)

The table has been created successfully:

In [355]:
dbUtils.runQuery('''
    SELECT column_name, data_type 
    FROM information_schema.columns
    WHERE table_name = '''+"'"+tableName+'''';
''')

Time used: 0.3105020523071289


,column_name,data_type
0,totalIndent,integer
1,numReflection,integer
2,numSnakes,integer
3,timestamp,integer
4,repoId,integer
5,loc,integer
6,cloc,integer
7,fileCount,integer
8,numMethods,integer
9,numLambdas,integer


Because of the pandas dataframe, data can easily be written to the database:

In [23]:
data.to_sql(tableName, schema='crm20', con=dbUtils.engine, if_exists='append', index=False)

And has successfully been created:

In [24]:
dbUtils.runQuery('''
    SELECT *
    FROM crm20.'''+tableName+'''
''')

Time used: 0.005092144012451172


,sha,parent,timestamp,repo_id,loc,cloc,file_count,num_methods,num_lambdas,num_comment_lines,num_reflection,num_snakes,total_indent
0,37bcca3d0bc7b1f03818ecf24b15951b444b4f76,71e5cb71cd634f5170a4285caf2636308d5eb999,1590332677,123456789,29844,25672,276,1627,226,4235,93,1741,57176
1,71e5cb71cd634f5170a4285caf2636308d5eb999,a49e5be01918f8012875befe18821676dc3ce98d,1590329536,123456789,29859,25687,276,1627,226,4233,93,1741,57269
2,a49e5be01918f8012875befe18821676dc3ce98d,54ac50b59b456cdc156b4aa18c7be17db1511950,1588277133,123456789,29849,25675,276,1627,226,4233,93,1741,57250
3,54ac50b59b456cdc156b4aa18c7be17db1511950,0c42536c5657adf2e910833cc72fea577d8ee1eb,1588096131,123456789,29825,25648,276,1627,226,4230,93,1741,57070
4,0c42536c5657adf2e910833cc72fea577d8ee1eb,81171bf7a031077d010518344a28b0d89c6f1e90,1588095924,123456789,29781,25610,276,1627,226,4223,93,1741,56867
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,411d51771ee6c714db13c23e8912086840b84f51,9e8faf6f0b516334451c53960dee001a6e9e42c4,1485386460,123456789,12972,10850,158,691,15,1454,31,117,25774
476,e36dd96a1201ad35578a5e5d65a5db02135f4614,0d893a471f932b921473f72563a540da46f15fa5,1484674213,123456789,12966,10836,155,687,15,1463,30,117,25851
477,0d893a471f932b921473f72563a540da46f15fa5,9e8faf6f0b516334451c53960dee001a6e9e42c4,1484673978,123456789,12966,10836,155,687,15,1463,30,117,25851
478,9e8faf6f0b516334451c53960dee001a6e9e42c4,e1ca54916ff570656eb1d40c13b7e58c10d8bc0a,1484305822,123456789,12922,10795,155,686,15,1466,30,117,25717


Mistakes will be made and experiments need to be undertaking, so cleaning up is necessary:<br>
Note: Because dropping a table does not return values, an error is thrown, this can be safely ignored for now 

In [25]:
dbUtils.runQuery('''
    DROP TABLE crm20.'''+tableName+'''
''')

ResourceClosedError: This result object does not return rows. It has been closed automatically.

## Create utility functions

All the results from the previous section can be extracted to some utility functions. These can later be extracted to an appropriate module.

At the core, the `runSuite` function stands: Calculating metrics for a repo and then writing them to table.

In [28]:
dataBaseSemaphore = multiprocessing.Semaphore()
def runSuite(repo):
    with io.capture_output() as output:
        data = repoAnalysis.calculateMetrics(repo, suite)
        with dataBaseSemaphore:
            data.to_sql(tableName, schema='crm20', con=dbUtils.engine, if_exists='append', index=False)
            dbUtils.engine.dispose()
    log(output)
    return len(data) > 0

For this, the table has to be created:

In [30]:
def createTable():
    columns = [Column('sha', String), Column('parent', String), Column('timestamp', Integer), Column('repoId', Integer)]
    columns = columns + list(map(lambda func: Column(func.__name__, Integer), suite))
    meta = MetaData(schema='crm20')
    table = Table(
        tableName, meta,
        *columns
    )
    meta.create_all(dbUtils.engine)
    dbUtils.engine.dispose()

Additionally, table deletion is needed for clean up of failed runs (which there were a lot). Exception handling is put around, because an error is returned as the querey does not return data

In [29]:
def deleteTable():
    try:
        dbUtils.runQuery('''
            DROP TABLE crm20.'''+tableName+'''
        ''')
    except:
        pass

Lastly, it had shown that for longer running processes, display sessions are not adequate to hold outputs. Therefore, output was written to a log file. This function was later extracted to the [dbUtils](dbUtils.py) module

In [27]:
logSemaphore = multiprocessing.Semaphore()

def log(text):
    with logSemaphore:
        with open('log.txt', 'a') as file:
            file.write('========= '+str(datetime.datetime.now())+' ==========\n'+str(text))

# First iteration

After preparing a first iteration of analysis, the first run started with approx. 200 users (100 in each group).

In [12]:
tableName = 'lb_results1'

In [9]:
repoLibrarian.setReposFolder('/mnt/brick/crm20/repos/')

'/mnt/brick/crm20/repos/'

Now all the results from the other sub-projects are brought in:<br>
The projects to analyze are taken from the [data explorer](DataExplorer.ipynb):

In [14]:
polyglotProjects = dbUtils.runQuery('''SELECT * FROM lb_polyglotProjects''')
controlgroupProjects = dbUtils.runQuery('''SELECT * FROM lb_controlgroupProjects''')
bothProjects = pandas.concat([polyglotProjects, controlgroupProjects], axis=0)

Time used: 0.37739133834838867


Time used: 0.36337733268737793


Repos are downloaded and managed with the help of the [repoLibrarian](repoLibrarian.py):<br>
For this, table rows must however be preprocessed: 

In [76]:
def rowToTuple(x):
    (index, row) = x
    split = row['url'].split('/')
    user = split[-2]
    project = split[-1]
    return (user, project, row['repo_id'])

Downloading repos takes longer time than expected and shows that there are many repos that are not accessible anymore. Many repos have shown to just have been renamed, however, implementing a github rename detection strategy is out of scope of this project. Also it is useful to filter the repositories by Java, so no unnecessary downloads occur.

In [16]:
repos = list(filter(lambda tupl: repoLibrarian.isJavaRepo(tupl[0], tupl[1]), map(rowToTuple, bothProjects.iterrows())))

Failed to check ('thiagoruis', 'dotNET-Grupo-2'): Reference at 'refs/heads/master' does not exist
Could not download repo "tananaev/traccar-client": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/tananaev/traccar-client.git /mnt/brick/crm20/repos/tananaev/traccar-client.git
  stderr: 'Cloning into bare repository '/mnt/brick/crm20/repos/tananaev/traccar-client.git'...
fatal: could not read Username for 'https://github.com': No such device or address
'
Failed to check ('tananaev', 'traccar-client'): Reference at 'refs/heads/master' does not exist
Could not download repo "topicusonderwijs/wicket-openid": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/topicusonderwijs/wicket-openid.git /mnt/brick/crm20/repos/topicusonderwijs/wicket-openid.git
  stderr: 'Cloning into bare repository '/mnt/brick/crm20/repos/topicusonderwijs/wicket-openid.git'...
fatal: could not read Username for 'https://github.com': No

To compare how many repos of the experiment groups can actually be downloaded and are Java repos: Approx. half

In [700]:
print(len(bothProjects))
print(len(repos))

1377
620


Then the run started. The commented line shows that there were precursive runs with the reduced local dataset (and random repo ids).<br>
The results of this run can be found in table `lb_results1` and their evaluation in the [Results_Iteration#1](Results_Iteration#1.ipynb) notebook. <br>
The log for this run can be found under [./results/log_first_data_run.txt](./results/log_first_data_run.txt).
It shows that approx. 2d were taken, which can surely be improved but which is good for a first large run.

The Evaluation of the results can be found in the [Results_Iteration#1](Results_Iteration#1.ipynb) notebook.

In [ ]:
deleteTable()
createTable()
#repos = list(map(lambda tupl: (*tupl, int.from_bytes(bytearray(str(tupl), 'utf-8'), byteorder='big', signed=False) % 10000000), repoLibrarian.managedRepos()))
start = time.time()
with Pool(int(multiprocessing.cpu_count()/4)) as pool:
    allMetrics = pool.map(runSuite, repos)
end = time.time()
dbUtils.log('Total Time used: '+str(end - start))

# Second iteration

The analysis of the data of the first run has shown several shortcomings that need to be addressed before scaling up:

Low severity:
* Camel case column names lead to errors or need to be escaped
* If the lifecycle analysis approach should be further pursued, last written date should be added to avoid warping the results of retired developers

High severity:
* `iter_commits` was missing the `--all` parameter so the analysis just ran on master branches - ignoring other branches!
* Using commit size information from the raw_patches table decreases our data set significantly
* The run took 2 days for 620 repositories. While this is already quite good, this makes it unfeasible to scale up by order of magnitude. If possible, opportunities for optimization should be taken

The camel case problem can easily be solved by renaming the functions used in the `repoAnalysis.suite` (e.g. `num_lambdas` instead of `numLambdas`).

As suggested, the lifeycycle problem can be solved by adding a column last_written to the `lb_languages` and dependent views. However, this second iteration should run on the same data as the first (to increase comparability of results). Recreating all views and accepting the losses therefore does not seem to be worth and will be postponed.

## Finding a new way to get commit changes information

In [42]:
testRepoId = ('tfox12', 'REST-Debugger')

In [43]:
repoLibrarian.hasRepo(*testRepoId)

True

In [44]:
from git.db import GitDB
from git.db import GitCmdObjectDB
from git import Repo 

In [77]:
testRepo = Repo.init(repoLibrarian.pathFor(*testRepoId), bare=True, odbt=GitCmdObjectDB)

### Use gitpython commit object functions

GitPython commit objects provide a stats attribute where diff information can be queried from.<br>
However, this is order of magnitude worse, so unfeasible.

In [46]:
start = time.time()
for commit in testRepo.iter_commits():
    for obj in commit.tree.traverse():
        if obj.type == 'blob' and obj.name.endswith('.java'):
            contentWithHeader = obj.data_stream.read().decode("CP437")#.decode("utf-8")
end = time.time()
print('Total Time used: '+str(end - start))

start = time.time()
res = []
for commit in testRepo.iter_commits():
    for obj in commit.tree.traverse():
        if obj.type == 'blob' and obj.name.endswith('.java'):
            contentWithHeader = obj.data_stream.read().decode("CP437")#.decode("utf-8")
    res.append(sum(map(lambda file: file[1]['lines'], (filter(lambda file: file[0].endswith('.java'), commit.stats.files.items())))))
end = time.time()
print('Total Time used: '+str(end - start))

Total Time used: 0.47133684158325195
Total Time used: 5.964926481246948


Gitpython commits can also diff on the fly with parent commits. This takes approximately the same time so it does not work either. 

In [52]:
start = time.time()
res = []
for commit in testRepo.iter_commits():
    for obj in commit.tree.traverse():
        if obj.type == 'blob' and obj.name.endswith('.java'):
            contentWithHeader = obj.data_stream.read().decode("CP437")#.decode("utf-8")
    if len(commit.parents) == 1:
        diff = testRepo.git.diff(commit.parents[0].hexsha, commit.hexsha, '--', numstat=True)
        # can be further parsed
end = time.time()
print('Total Time used: '+str(end - start))

Total Time used: 6.147816896438599


### Dissect gitpython

- Using gitpython interface methods works, but the needed time is far too bad
- It might be worth to look into internals of gitpython, iterating over commits (and all of them files!) is fast
- Does this mean, gitpython uses a fast way to apply deltas? (when iterating over files is fast)
- For this I looked up the gitpython sources at https://github.com/gitpython-developers/GitPython

Aggregated Insights:
- Iterating over objects of commit needs irritatingly little time -> iter_commits needs 99% of time
- `iter_commits` iterates over revlist and creates commit for each; subcalls are
    - `testRepo.git.rev_list(testRepo.head.commit))`
    - `Commit._iter_from_process_or_stream`
- The time is spend in `cmd.py>>_call_process` which is aliased with `__getattr__` --> this means gets called on attribute getting
    - The call stack is `repo.iter_commits -> Commit.iter_items -> repo.git.rev_list` so it is the rev-list call that takes all the time
    - Repo.git is of type `GitCommandWrapperType` which is `cmd.py>>Git`
    - However, diff information is not included in the rev-list call (which just translates to a git command call)
- `_iter_from_process_or_stream` parses hexshas from a stream and creates commits out of them
- That means the file contents come directly from `data_stream` which calls odb.stream(binsha)
- So file contents are accessed directly via a hash - no way for me to get the diff that way...

In [64]:
%%timeit -n 20
commits = testRepo.iter_commits()

81.9 ms ± 1.62 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [57]:
commits = testRepo.iter_commits()

In [65]:
%%timeit -n 20
for commit in commits:
    for obj in commit.tree.traverse(predicate=lambda obj, depth: obj.type == 'blob' and obj.name.endswith('.java')):
        contentWithHeader = obj.data_stream.read().decode("CP437")#.decode("utf-8")

171 ns ± 86.3 ns per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [69]:
%prun -l 15 -s cumulative testRepo.iter_commits()

         885 function calls (882 primitive calls) in 0.092 seconds

   Ordered by: cumulative time
   List reduced from 156 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.092    0.092 {built-in method builtins.exec}
        1    0.000    0.000    0.092    0.092 <string>:1(<module>)
        1    0.000    0.000    0.091    0.091 base.py:504(iter_commits)
        1    0.000    0.000    0.089    0.089 commit.py:186(iter_items)
        1    0.000    0.000    0.089    0.089 cmd.py:542(<lambda>)
        1    0.000    0.000    0.089    0.089 cmd.py:940(_call_process)
        1    0.000    0.000    0.089    0.089 cmd.py:580(execute)
        1    0.000    0.000    0.087    0.087 subprocess.py:732(__init__)
        1    0.001    0.001    0.087    0.087 subprocess.py:1550(_execute_child)
        1    0.063    0.063    0.063    0.063 {built-in method _posixsubprocess.fork_exec}
        1    0.020    0.020    0.

In [469]:
from git.objects import Commit
proc = testRepo.git.rev_list(testRepo.head.commit, as_process=True)
print(proc)
commits = Commit._iter_from_process_or_stream(testRepo, proc)
for commit in commits:
    print(commit.hexsha)

ea393258a29f93539e371156cb77ea8eb2755b31
8a94a36e5448dd83ef4dafc76cc7650e8b20f791
cf84234e66b5a9940a3654b3bf6c17e8ec23e03c
468a8e7939c7328d3af5b95fe8c6832966df1b52
e78ae0397b209312ba80b709ba6996b558bb13bf
d85a3642c97d44d6c4b0bcd145c68f420b3c2d96
9dcfe3d3e498dfb06ae0f8c00dd1eda72bda070d
81a242f338713512b8d6b52d2a228a2be2f70d27
6fcc68689f3772ecaa1a6a011510ff26d7ee48d6
3f93813edef9786f6cb8ced6a7441783da3f8884
34503c976c764ce103baf2c10721ffa3656f20f8
1b3e586b70006b9a21d9a35c5029c1e48cd95837
a198e51031eadf8a013cc3711e37656f9edbe27b
3682dc8523e86a00251a237258047429c1bb6cf9
58c4dcce5ee8f618c0b12fd9053a79df8558b5a6
0f4e0bb6b43d088baca4e874d2641c6bc3e427cf
49f326042cf26b2b9ead9b675b6a830704092fa3
5be9b9fa33c509896fc1ad3c0e9d09680135153f
e7576bd7f72e7e5286d23bbad167cf3f394bed9c
ed4dd39de749f287ad3036832ebcf24126140b72
6c51645d3341163b45552ccfc47c162c8a0d0dc8
377ccc918b4b7d3f390bcba84777e1555e501622
0c5ddda62b6911aaec177be1f0e3d028dd94f94c
9c6d5dd48018dc9d72d952913f3c3f9c92530574
4048f135cdc7a3de

### Manual diff approach

As I now know that gitpython subprocesses are what costs most of the time, there are alternative approaches: Diffing manually can be fast and avoids cmds.<br>
Therefore, I took an implementation of the myers algorithm (which is also used in git) Source: https://gist.github.com/tonyg/2361e3bfe4e92a1fc6f7 <br>
Despite sounding like a bad idea this approach actually is quite good. One problem however is that the algorithm provides slightly different results than the git internal one. 

In [72]:
# Source: https://gist.github.com/tonyg/2361e3bfe4e92a1fc6f7

# Copyright (c) 2015 Tony Garnock-Jones <tonyg@leastfixedpoint.com>
#
# Permission is hereby granted, free of charge, to any person
# obtaining a copy of this software and associated documentation files
# (the "Software"), to deal in the Software without restriction,
# including without limitation the rights to use, copy, modify, merge,
# publish, distribute, sublicense, and/or sell copies of the Software,
# and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
#
# The above copyright notice and this permission notice shall be
# included in all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
# EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
# MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS
# BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN
# ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Text diff algorithm after Myers 1986 and Ukkonen 1985, following
# Levente Uzonyi's Squeak Smalltalk implementation at
# http://squeaksource.com/DiffMerge.html
#
# E. W. Myers, "An O(ND) difference algorithm and its variations,"
# Algorithmica, vol. 1, no. 1-4, pp. 251-266, Nov. 1986.
#
# E. Ukkonen, "Algorithms for approximate string matching," Inf.
# Control, vol. 64, no. 1-3, pp. 100-118, Jan. 1985.

def longest_common_subsequence(xs, ys):
    totallen = len(xs) + len(ys)
    frontier = [0] * (2 * totallen + 1)
    candidates = [None] * (2 * totallen + 1)
    for d in range(totallen + 1):
            for k in range(-d, d+1, 2):
                    if k == -d or (k != d and frontier[totallen + k - 1] < frontier[totallen + k + 1]):
                            index = totallen + k + 1
                            x = frontier[index]
                    else:
                            index = totallen + k - 1
                            x = frontier[index] + 1
                    y = x - k
                    chain = candidates[index]
                    while x < len(xs) and y < len(ys) and xs[x] == ys[y]:
                            chain = ((x, y), chain)
                            x = x + 1
                            y = y + 1
                    if x >= len(xs) and y >= len(ys):
                            result = []
                            while chain:
                                    result.append(chain[0])
                                    chain = chain[1]
                            result.reverse()
                            return result
                    frontier[totallen + k] = x
                    candidates[totallen + k] = chain

def myer_diff(xs, ys):
        i = -1
        j = -1
        matches = longest_common_subsequence(xs, ys)
        matches.append((len(xs), len(ys)))
        result = []
        for (mi, mj) in matches:
                if mi - i > 1 or mj - j > 1:
                        result.append((i + 1, mi - i - 1, j + 1, mj - j - 1))
                i = mi
                j = mj
        return result

In [79]:
start = time.time()
for commit in testRepo.iter_commits():
    if len(commit.parents) == 1:
        #print(commit.hexsha)
        for obj in commit.tree.traverse(predicate=lambda obj, depth: obj.type == 'blob' and obj.name.endswith('.java')):
            try:
                obj2 = commit.parents[0].tree / obj.path
                content1 = obj.data_stream.read().decode("CP437")#.decode("utf-8")
                content2 = obj2.data_stream.read().decode("CP437")#.decode("utf-8")
                if not (content1 == content2):
                    #print('\tchanged '+obj.path)
                    diff = myer_diff(content1.split('\n'), content2.split('\n'))
                    additions = sum(map(lambda change: change[1], diff))
                    deletions = sum(map(lambda change: change[3], diff))
                    #print(str(additions)+'\t'+str(deletions)+'\t'+obj.path)
            except KeyError:
                pass
                #print('\tadded '+obj.path)
        #print('---\n'+testRepo.git.diff(commit.parents[0].hexsha, commit.hexsha, '--', numstat=True))
        #print(testRepo.git.diff(commit.parents[0].hexsha, commit.hexsha))
        #print('\n')
end = time.time()
print('Total Time used: '+str(end - start))

Total Time used: 0.5856478214263916


### git log approach

We know that iter_commits takes 90% of time, because it calls rev-list. Calling a git command for every commit or even every file increases cost by magnitude. So I opened a [stackoverflow thread](https://stackoverflow.com/questions/63115815/fast-way-to-get-number-of-changed-deleted-lines-per-subset-of-files-per-commit) which resulted in a tip to use `git log --numstat`. 

I used this approach to generate one machine readable full history all files commit diff log:  

In [84]:
import subprocess
start = time.time()
print(subprocess.check_output('git -C '+repoLibrarian.pathFor(*testRepoId)+' log --numstat --format=//%H', shell=True).decode('utf-8'))
end = time.time()
print('Total Time used: '+str(end - start))

//ea393258a29f93539e371156cb77ea8eb2755b31

144	0	web/lib/debugger.js
//8a94a36e5448dd83ef4dafc76cc7650e8b20f791

1	1	src/capstone/wrapper/GdbWrapper.java
11	5	web/test.html
//cf84234e66b5a9940a3654b3bf6c17e8ec23e03c

22	2	src/capstone/wrapper/GdbWrapper.java
//468a8e7939c7328d3af5b95fe8c6832966df1b52

2	1	makefile
11	0	src/capstone/daemon/DaemonHandler.java
1	1	src/capstone/wrapper/PdbWrapper.java
2	0	test/capstone/wrapper/GdbWrapperTest.java
20	108	web/test.html
//e78ae0397b209312ba80b709ba6996b558bb13bf

10	6	src/capstone/daemon/DaemonHandler.java
231	0	src/capstone/wrapper/PdbWrapper.java
129	123	src/capstone/wrapper/Wrapper.java
31	0	test/capstone/wrapper/PdbWrapperTest.java
//d85a3642c97d44d6c4b0bcd145c68f420b3c2d96

1	0	.gitignore
//9dcfe3d3e498dfb06ae0f8c00dd1eda72bda070d

5	1	src/capstone/wrapper/Wrapper.java
//81a242f338713512b8d6b52d2a228a2be2f70d27

12	2	src/capstone/wrapper/GdbWrapper.java
1	1	src/capstone/wrapper/Wrapper.java
//6fcc68689f3772ecaa1a6a011510ff26d7ee48d6

1	

Adding this to our current approach will double the run time, which would be ok. But, with all the knowledge about the inner workings of gitpython, rev-log is just used as text input to get all commit shas - this data is also included in the log! If we replace that with a git log --numstats, it's not too much slower than before but we get detailed diff information - we can filter that for java files and sum up and even pursue a diff-based approach instead of calculating the whole project metrics every time.

In [87]:
from git.util import hex_to_bin
from git import Commit, Repo

repo = Repo.init('./repos/bptlab/scylla.git', bare=True)

#New code in comparison:
start = time.time()
log = repo.git.log('--numstat', '--format=//%H', '--all')
blocks = log.split('//')[1:]
commits = map(lambda block: block.split('\n')[0], blocks)
for hexsha in commits:
    commit = Commit(repo, hex_to_bin(hexsha))
    for obj in commit.tree.traverse(predicate=lambda obj, depth: obj.type == 'blob' and obj.name.endswith('.java')):
        content = obj.data_stream.read().decode("CP437")
end = time.time()
print('Total Time used for new method: '+str(end - start))

#Old code in comparison:
start = time.time()
log = repo.git.log('--numstat', '--format=//%H', '--all')
blocks = log.split('//')[1:]
commits = map(lambda block: block.split('\n')[0], blocks)
for commit in repo.iter_commits():
    for obj in commit.tree.traverse(predicate=lambda obj, depth: obj.type == 'blob' and obj.name.endswith('.java')):
        content = obj.data_stream.read().decode("CP437")
end = time.time()
print('Total Time used for old method: '+str(end - start))

Total Time used for new method: 13.67722201347351
Total Time used for old method: 9.329468011856079


## Delta based analysis approach 

With the insights from the previous section, a great opportunity has risen: Only analyzing those files of a commit that have actually changed. The implications of this tuning are huge: the time complexity of the analysis run is reduced from quadratic in repository size (number of commits times number of files) to linear (number of commits times (more or less constant) commit size). 

To actually implement this, several sub functions must be implemented: `block_to_stats` extracts the changed java files and sum of changes from blocks of the commit log. `file_contents` is the new way to access files outside of a commit tree iterator.

In [88]:
def safeToInt(string):
    return 0 if string == '-' else int(string)

def block_to_stats(block):
    lines = block.split('\n')
    header = lines[0]
    lines = filter(lambda line: line.endswith('.java'), lines)
    changed_files = list(map(lambda line: line.split('\t'), lines))
    #changed_files = list(changed_files)
    additions = sum(map(lambda file: safeToInt(file[0]), changed_files))
    deletions = sum(map(lambda file: safeToInt(file[1]), changed_files))
    return (header, (changed_files, additions, deletions))

def file_contents(tree, path):
    try: 
        obj = tree / path
        return obj.data_stream.read().decode("CP437")
    except KeyError:
        return ''

The following run demonstrates how the new approach can be used; the file contents and parent commit file contents are queried to simulate normal load.<br>
This is "only" twice as fast as the original approach, but the effect turned out to be huge for larger repos.

In [91]:
testRepo = Repo.init('./repos/bptlab/scylla.git', bare=True)
start = time.time()
log = testRepo.git.log('--numstat', '--format=//%H', '--all')
#print(log)
commits = log.split('//')[1:]
changes = map(block_to_stats, commits)

for hexsha, change in changes:
    commit = Commit(testRepo, hex_to_bin(hexsha))
    if len(commit.parents) == 1:
        changed_files, additions, deletions = change
        for added, removed, file in changed_files:
            content1 = file_contents(commit.tree, file)
            content2 = file_contents(commit.parents[0].tree, file)

end = time.time()
print('Total Time used for new method: '+str(end - start))

Total Time used for new method: 5.317599773406982


In [92]:
start = time.time()
for commit in repo.iter_commits('--all'):
    for obj in commit.tree.traverse(predicate=lambda obj, depth: obj.type == 'blob' and obj.name.endswith('.java')):
        content = obj.data_stream.read().decode("CP437")
end = time.time()
print('Total Time used for old method: '+str(end - start))

Total Time used for old method: 10.103673458099365


Another test run with a bigger repository:

In [93]:
testRepoTuple = ('brockn', 'incubator-parquet-mr', 11108627)

In [672]:
repoAnalysis.calculateMetrics(testRepoTuple)

Time used for ('brockn', 'incubator-parquet-mr', 11108627): 435.4707381725311


,sha,parent,timestamp,repoId,additions,deletions,loc,cloc,fileCount,numMethods,numLambdas,numCommentLines,numReflection,numSnakes,totalIndent
0,69ba4844730426a212c609facd93b33bf6692b3a,be1222ef4a3260ddcf516d73c6ceecd144a134cb,1412699955,11108627,0,0,71466,56538,489,5214,11,6229,360,1966,54355.75
1,be1222ef4a3260ddcf516d73c6ceecd144a134cb,da9129927bce90feb6d2860745263f4d74d0dfa8,1412198064,11108627,0,0,71461,56534,489,5214,11,6229,360,1964,54351.75
2,da9129927bce90feb6d2860745263f4d74d0dfa8,0b17cbee9541998df66d33c8a99b675ced80d9aa,1412196285,11108627,0,0,71454,56527,489,5214,11,6229,360,1960,54340.75
3,0b17cbee9541998df66d33c8a99b675ced80d9aa,bf20abbf4825fa5892d8e15c066e768671a39289,1412017203,11108627,0,0,71157,56235,489,5197,11,6229,360,1849,53845.25
4,bf20abbf4825fa5892d8e15c066e768671a39289,3a082e8e390898646c094d20f4ec1eeba45b79ac,1411688756,11108627,0,0,71153,56231,489,5197,11,6229,360,1849,53840.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,dbbf9443ffe219862c9744806f34deb2606d21c7,f3cdad30930c2cd73404804cf4ceeca237509a2c,1346885521,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50
1831,24028799609ef606359d6bee39acd34d8d67a067,f3cdad30930c2cd73404804cf4ceeca237509a2c,1346454047,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50
1832,f3cdad30930c2cd73404804cf4ceeca237509a2c,a8c10efccf35977193cab80b0f17d6a2f7d066d9,1346452819,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50
1833,a8c10efccf35977193cab80b0f17d6a2f7d066d9,576c709724551a5122ae9b9e314b6c400f5f778d,1346452652,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50


Using the git log approach only to determine the commits still results in a comparable run time - while providing the needed diff information

In [673]:
def alternativeCalculateMetrics(repoTuple, metricSuite=repoAnalysis.metricSuite):
    (user, project, repoId) = repoTuple
    repo = repoLibrarian.getRepo(user, project)
    columns = ['sha', 'parent', 'timestamp', 'repoId', 'additions', 'deletions'] + list(map(lambda fun: fun.__name__, metricSuite))
    results = []
    try:
        start = time.time()
        log = repo.git.log('--numstat', '--format=//%H', '--all')
        blocks = log.split('//')[1:]
        commits = map(lambda block: block.split('\n')[0], blocks)
        for hexsha in commits:
            commit = Commit(repo, hex_to_bin(hexsha))
            results.append(repoAnalysis.metricsForCommit(commit, metricSuite, repoId))
        df = pandas.DataFrame(results, columns=columns)
        end = time.time()
        print('Time used for '+str(repoTuple)+': '+str(end - start))
        return df
    except Exception as e:
        print('Failed to analyze '+str(repoTuple)+': '+str(e))
        return []

In [674]:
alternativeCalculateMetrics(testRepoTuple)

Time used for ('brockn', 'incubator-parquet-mr', 11108627): 439.26439571380615


,sha,parent,timestamp,repoId,additions,deletions,loc,cloc,fileCount,numMethods,numLambdas,numCommentLines,numReflection,numSnakes,totalIndent
0,69ba4844730426a212c609facd93b33bf6692b3a,be1222ef4a3260ddcf516d73c6ceecd144a134cb,1412699955,11108627,0,0,71466,56538,489,5214,11,6229,360,1966,54355.75
1,be1222ef4a3260ddcf516d73c6ceecd144a134cb,da9129927bce90feb6d2860745263f4d74d0dfa8,1412198064,11108627,0,0,71461,56534,489,5214,11,6229,360,1964,54351.75
2,da9129927bce90feb6d2860745263f4d74d0dfa8,0b17cbee9541998df66d33c8a99b675ced80d9aa,1412196285,11108627,0,0,71454,56527,489,5214,11,6229,360,1960,54340.75
3,0b17cbee9541998df66d33c8a99b675ced80d9aa,bf20abbf4825fa5892d8e15c066e768671a39289,1412017203,11108627,0,0,71157,56235,489,5197,11,6229,360,1849,53845.25
4,bf20abbf4825fa5892d8e15c066e768671a39289,3a082e8e390898646c094d20f4ec1eeba45b79ac,1411688756,11108627,0,0,71153,56231,489,5197,11,6229,360,1849,53840.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1830,dbbf9443ffe219862c9744806f34deb2606d21c7,f3cdad30930c2cd73404804cf4ceeca237509a2c,1346885521,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50
1831,24028799609ef606359d6bee39acd34d8d67a067,f3cdad30930c2cd73404804cf4ceeca237509a2c,1346454047,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50
1832,f3cdad30930c2cd73404804cf4ceeca237509a2c,a8c10efccf35977193cab80b0f17d6a2f7d066d9,1346452819,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50
1833,a8c10efccf35977193cab80b0f17d6a2f7d066d9,576c709724551a5122ae9b9e314b6c400f5f778d,1346452652,11108627,0,0,2858,2502,40,323,3,71,7,2,2190.50


Using the approach to its full extend sped up to a factor of more than 25

In [95]:
def deltaMetricsForCommit(commit, metricSuite, repoId, change):
    
    resultTuple = {
        'sha' : commit.hexsha,
        'parent' : commit.parents[-1].hexsha if len(commit.parents) == 1 else None,
        'timestamp' : commit.committed_date,
        'repoId' : repoId
    }    
    for metricFunction in metricSuite:
        resultTuple[metricFunction.__name__] = 0
    
    changed_files, additions, deletions = change
    resultTuple['additions'] = additions
    resultTuple['deletions'] = deletions
    for added, removed, file in changed_files:
        
        contentWithHeader = file_contents(commit.tree, file)
        content = repoAnalysis.removeHeader(contentWithHeader)
        contentWithoutStrings = repoAnalysis.stringRemoveRegex.sub("\"...\"", content)
        contentWithoutComments = repoAnalysis.commentRegex.sub("/*...*/", contentWithoutStrings)
        for metricFunction in metricSuite:
            metric = metricFunction(content=content, contentWithHeader=contentWithHeader, contentWithoutComments=contentWithoutComments)
            resultTuple[metricFunction.__name__] = resultTuple[metricFunction.__name__] + metric
            
        contentWithHeader = file_contents(commit.parents[0].tree, file)
        content = repoAnalysis.removeHeader(contentWithHeader)
        contentWithoutStrings = repoAnalysis.stringRemoveRegex.sub("\"...\"", content)
        contentWithoutComments = repoAnalysis.commentRegex.sub("/*...*/", contentWithoutStrings)
        for metricFunction in metricSuite:
            metric = metricFunction(content=content, contentWithHeader=contentWithHeader, contentWithoutComments=contentWithoutComments)
            resultTuple[metricFunction.__name__] = resultTuple[metricFunction.__name__] - metric
            
    return resultTuple
    

def calculateDeltaMetrics(repoTuple, metricSuite=repoAnalysis.metricSuite):
    (user, project, repoId) = repoTuple
    repo = repoLibrarian.getRepo(user, project)
    columns = ['sha', 'parent', 'timestamp', 'repo_id', 'additions', 'deletions'] + list(map(lambda fun: fun.__name__, metricSuite))
    results = []
    try:
        start = time.time()
        log = repo.git.log('--numstat', '--format=//%H', '--all')
        commits = log.split('//')[1:]#First 'part'
        changes = map(block_to_stats, commits)

        for hexsha, change in changes:
            commit = Commit(repo, hex_to_bin(hexsha))
            if len(commit.parents) == 1:
                results.append(deltaMetricsForCommit(commit, metricSuite, repoId, change))
                
        df = pandas.DataFrame(results, columns=columns)
        end = time.time()
        print('Time used for '+str(repoTuple)+': '+str(end - start))
        return df
    except Exception as e:
        print('Failed to analyze '+str(repoTuple)+': '+str(e))
        return []

In [96]:
calculateDeltaMetrics(testRepoTuple)

Time used for ('brockn', 'incubator-parquet-mr', 11108627): 16.90212655067444


,sha,parent,timestamp,repo_id,additions,deletions,loc,cloc,file_count,num_methods,num_lambdas,num_comment_lines,num_reflection,num_snakes,total_indent
0,69ba4844730426a212c609facd93b33bf6692b3a,be1222ef4a3260ddcf516d73c6ceecd144a134cb,1412699955,NaN,7,2,5,4,0,0,0,0,0,2,4.00
1,be1222ef4a3260ddcf516d73c6ceecd144a134cb,da9129927bce90feb6d2860745263f4d74d0dfa8,1412198064,NaN,19,12,7,7,0,0,0,0,0,4,11.00
2,da9129927bce90feb6d2860745263f4d74d0dfa8,0b17cbee9541998df66d33c8a99b675ced80d9aa,1412196285,NaN,346,49,297,292,0,17,0,0,0,111,495.50
3,0b17cbee9541998df66d33c8a99b675ced80d9aa,bf20abbf4825fa5892d8e15c066e768671a39289,1412017203,NaN,34,30,4,4,0,0,0,0,0,0,4.50
4,bf20abbf4825fa5892d8e15c066e768671a39289,3a082e8e390898646c094d20f4ec1eeba45b79ac,1411688756,NaN,34,1,33,33,0,11,0,0,0,0,15.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,5fb63e9a0743a23735f5b6043abd202a85173202,7ed952872d68eacc02d5a13e6b6ded4cf8babc55,1347128023,NaN,304,146,158,149,0,30,2,32,1,0,83.75
1450,dbbf9443ffe219862c9744806f34deb2606d21c7,f3cdad30930c2cd73404804cf4ceeca237509a2c,1346885521,NaN,0,0,0,0,0,0,0,0,0,0,0.00
1451,24028799609ef606359d6bee39acd34d8d67a067,f3cdad30930c2cd73404804cf4ceeca237509a2c,1346454047,NaN,0,0,0,0,0,0,0,0,0,0,0.00
1452,f3cdad30930c2cd73404804cf4ceeca237509a2c,a8c10efccf35977193cab80b0f17d6a2f7d066d9,1346452819,NaN,0,0,0,0,0,0,0,0,0,0,0.00


The above developed functions have been integrated into the [repoAnalysis](repoAnalysis.py) module.

## Analysis Run

With the listed problems solved and a new approach developed, the second run can now start. Because of the big changes, it is interesting to run this analyis on the same data scale (and therefore developers) as the first one. Therefore, the approaches can be compared. 

In [107]:
tableName = 'lb_results2'

### Utility function updates

Create table had to be revised, as firstly additional columns `additions` and `deletions` had been introduced (these were formerly extracted from the table, now directly calculated during analysis) and secondly column names had been switched from camel to snake case, as there were errors when running queries on attributes in camel case.

In [38]:
def createTable():
    columns = [Column('sha', String), Column('parent', String), Column('timestamp', Integer), Column('repo_id', Integer), Column('additions', Integer), Column('deletions', Integer)]
    columns = columns + list(map(lambda func: Column(func.__name__, Integer), suite))
    meta = MetaData(schema='crm20')
    table = Table(
        tableName, meta,
        *columns
    )
    meta.create_all(dbUtils.engine)
    dbUtils.engine.dispose()

Writing to the database gained it's own function for better decomposition

In [39]:
dataBaseSemaphore = multiprocessing.Semaphore()
def writeDataToDb(data):
    with dataBaseSemaphore:
        data.to_sql(tableName, schema='crm20', con=dbUtils.engine, if_exists='append', index=False)
        dbUtils.engine.dispose()

Suite running was changed from the old total-count-calculation to calculating deltas only

In [40]:
def runDeltaSuite(repo):
    with io.capture_output() as output:
        data = repoAnalysis.calculateDeltaMetrics(repo, suite)
        writeDataToDb(data)
    dbUtils.log(output)
    return len(data) > 0

### Prepare the repos

In [12]:
repoLibrarian.setReposFolder('/mnt/brick/crm20/repos/')

'/mnt/brick/crm20/repos/'

In [13]:
polyglotProjects = dbUtils.runQuery('''SELECT * FROM lb_polyglotProjects''', mute=True)
controlgroupProjects = dbUtils.runQuery('''SELECT * FROM lb_controlgroupProjects''', mute=True)
bothProjects = pandas.concat([polyglotProjects, controlgroupProjects], axis=0)

Time used: 0.4349958896636963
Time used: 0.34889984130859375


In [24]:
start = time.time()
repos = list(filter(lambda tupl: repoLibrarian.isJavaRepo(tupl[0], tupl[1]), map(rowToTuple, bothProjects.iterrows())))
end = time.time()
dbUtils.log('Total Time used: '+str(end - start))

Failed to check ('thiagoruis', 'dotNET-Grupo-2'): Reference at 'refs/heads/master' does not exist
Failed to check ('tananaev', 'traccar-client'): Reference at 'refs/heads/master' does not exist
Failed to check ('topicusonderwijs', 'wicket-openid'): Reference at 'refs/heads/master' does not exist
Failed to check ('dav009', 'graficadora-tweets'): Reference at 'refs/heads/master' does not exist
Failed to check ('dav009', 'MWE-DictionaryExtractor'): Reference at 'refs/heads/master' does not exist
Failed to check ('OpenGamma', 'OG-RStats'): Reference at 'refs/heads/master' does not exist


KeyboardInterrupt: 

In [25]:
print(len(bothProjects))
print(len(repos))

1377
620


### Dry test to see if everything is up

In [26]:
dbUtils.log('test')

In [21]:
repos[0]

('twitter', 'scalding', 1182)

In [27]:
deleteTable()
createTable()
runDeltaSuite(repos[0])
display(Audio('./beep.mp3', autoplay=True))

(The results have been checked by simply running a database query)

### Let's go

In [ ]:
deleteTable()
createTable()
start = time.time()
with Pool(int(multiprocessing.cpu_count()*3/4)) as pool:
    allMetrics = pool.map(runDeltaSuite, repos)
end = time.time()
dbUtils.log('Total Time used: '+str(end - start))

The run log can be found under [./results/log_second_data_run.txt](./results/log_second_data_run.txt). 
It shows that this run analyzed the 620 repositories in 27 minutes ... that is an improvement by factor 106. This exceeded my expectations by far and motivates to scale up a lot for the next run.

Evaluation of the results can be found in the [Results_Iteration#2](Results_Iteration#2.ipynb) notebook.

## Emergency Recovery

Unfortunately, the `lb_results2` table has been lost. The set of polyglots used as input for this table could, against the expectations, not be reconstructed by simply filtering the extended set by `MOD(user_id, 113) = 5`. Therefore, a full recovery run has been started that hopefully yields the same results. Parallely, at least the relevant (analyzable) polyglot users could be recovered from memory in the [evaluation](Results_Iteration#2.ipynb) notebook and are used to reconstruct the relevant commit data.

First step is to reconstruct the basic tables, especially language information per author.

In [ ]:
dbUtils.runQuery('''
    CREATE MATERIALIZED VIEW crm20.temp_languages AS (
        WITH temp_sample_users AS (
            SELECT id 
            FROM ght.users 
            WHERE MOD(id, 113) = 5 
            AND type = 'USR'
        ),

        temp_projects AS (
            SELECT DISTINCT repo_id
            FROM temp_sample_users, ght.project_members 
            WHERE id = user_id
        ),

        temp_all_commits AS (
            SELECT commit_id 
            FROM ght.project_commits, temp_projects
            WHERE project_id = repo_id
        ),

        temp_commits AS (
            SELECT commit_id, sha, author_id, created_at, project_id 
            FROM temp_all_commits, ght.commits
            WHERE commit_id = id AND author_id IN (SELECT * FROM temp_sample_users)
        ),

        temp_changes AS (
            SELECT commit_id, temp_commits.sha, SUM(changes-deletions) AS loc, SUBSTRING(name from '\.([^\.]*)$') AS language
            FROM temp_commits, ght.raw_patches
            WHERE temp_commits.sha = raw_patches.sha
            GROUP BY commit_id, temp_commits.sha, language
        )

        SELECT author_id, language, SUM(loc) AS loc, MIN(created_at) as first_written
        FROM temp_commits, temp_changes
        WHERE temp_commits.commit_id = temp_changes.commit_id
        GROUP BY author_id, language
    );

    SELECT * FROM crm20.temp_languages
''')

dbUtils.log('lets see', 'resolved.txt')

Then java users and polyglot and control subgroups are determined

In [68]:
dbUtils.runQuery('''
    DROP VIEW IF EXISTS crm20.temp_java_users;
    
    CREATE VIEW crm20.temp_java_users AS (
        SELECT *
        FROM crm20.temp_languages
        WHERE language = 'java' AND loc > 3000
    );
        
    SELECT * FROM crm20.temp_java_users
''', mute=True)

Time used: 0.12581706047058105


,author_id,language,loc,first_written
0,118,java,4458.0,2014-07-12 13:58:27
1,1361,java,52662.0,2014-08-23 09:58:05
2,4073,java,5481.0,2014-08-14 02:10:11
3,5768,java,191889.0,2010-08-04 18:11:50
4,10401,java,4762.0,2012-08-30 09:36:51
...,...,...,...,...
1361,49736842,java,3194.0,2019-03-01 08:29:23
1362,49975159,java,12942.0,2019-03-18 09:09:26
1363,50518011,java,11297.0,2019-04-09 18:20:00
1364,50573155,java,72624.0,2019-04-10 13:08:48


In [70]:
dbUtils.runQuery('''
        CREATE MATERIALIZED VIEW crm20.temp_polyglots AS (
            SELECT temp_java_users.author_id, temp_java_users.loc AS javaLoC,  temp_java_users.first_written as javaStart, ARRAY_AGG(temp_languages.language) AS secondary_languages, ARRAY_AGG(temp_languages.loc) AS secLoC, ARRAY_AGG(temp_languages.first_written) as secStart
            FROM crm20.temp_java_users, crm20.temp_languages
            WHERE temp_java_users.author_id = temp_languages.author_id 
            AND (temp_languages.language = 'py' OR temp_languages.language = 'js') AND temp_languages.loc > 1000
            GROUP BY temp_java_users.author_id, temp_java_users.loc, temp_java_users.first_written
            HAVING temp_java_users.loc / 2 > ALL(ARRAY_AGG(temp_languages.loc))
            AND temp_java_users.first_written < ALL(ARRAY_AGG(temp_languages.first_written))
        );
        SELECT * FROM crm20.temp_polyglots
''', mute=True)

Time used: 0.21452021598815918


,author_id,javaloc,javastart,secondary_languages,secloc,secstart
0,55149,48533.0,2012-08-18 19:14:30,[js],[13340],[2015-08-10 01:20:42]
1,61025,231507.0,2010-12-17 19:50:34,[js],[1494],[2013-10-09 16:20:00]
2,89727,90236.0,2012-03-01 13:56:58,[js],[32537],[2013-02-10 15:10:56]
3,100010,4449.0,2013-08-21 21:01:11,[js],[1214],[2014-05-16 19:11:29]
4,100236,143991.0,2010-04-11 12:32:30,[js],[42598],[2015-01-31 21:20:39]
...,...,...,...,...,...,...
103,41410889,11749.0,2018-04-02 00:46:50,[js],[1263],[2018-04-13 04:04:22]
104,45852806,4813.0,2018-10-23 03:57:45,[py],[1928],[2018-11-09 02:44:09]
105,45985807,69531.0,2018-10-23 08:41:21,[js],[29569],[2018-10-23 08:57:52]
106,46354752,12717.0,2018-10-08 22:59:11,[js],[2197],[2018-10-09 13:36:55]


In [72]:
dbUtils.runQuery('''
        DROP MATERIALIZED VIEW IF EXISTS crm20.temp_controlgroup;
        CREATE MATERIALIZED VIEW crm20.temp_controlgroup AS (
            SELECT temp_java_users.author_id, temp_java_users.loc AS javaLoC,  temp_java_users.first_written as javaStart
            FROM crm20.temp_java_users
            WHERE author_id NOT IN (SELECT author_id FROM crm20.temp_languages WHERE (language = 'py' OR language = 'js'))
            AND MOD(author_id, 4) = 3 
            AND loc > 5000
        );
        SELECT * FROM crm20.temp_controlgroup
''', mute=True)

Time used: 0.2006971836090088


,author_id,javaloc,javastart
0,16503,106978.0,2010-09-30 11:48:03
1,34131,174060.0,2010-07-22 11:55:16
2,99671,20371.0,2012-07-14 12:27:07
3,153911,5064.0,2013-11-19 15:26:37
4,338779,13707.0,2012-07-09 07:15:19
...,...,...,...
101,46946759,10000.0,2019-05-10 07:02:55
102,49175119,16300.0,2019-02-03 02:28:33
103,49975159,12942.0,2019-03-18 09:09:26
104,50518011,11297.0,2019-04-09 18:20:00


In [73]:
display(dbUtils.runQuery('''
    CREATE VIEW crm20.temp_polyglotProjects AS (
        SELECT DISTINCT repo_id, url, name
        FROM temp_polyglots, ght.project_members, ght.projects
        WHERE author_id = user_id
        AND repo_id = id
    );
    SELECT * FROM  crm20.temp_polyglotProjects
''', mute=True))
display(dbUtils.runQuery('''
    CREATE VIEW crm20.temp_controlgroupProjects AS (
        SELECT DISTINCT repo_id, url, name
        FROM temp_controlgroup, ght.project_members, ght.projects
        WHERE author_id = user_id
        AND repo_id = id
    );
    SELECT * FROM crm20.temp_controlgroupProjects
''', mute=True))

Time used: 7.6404218673706055


,repo_id,url,name
0,1182,https://api.github.com/repos/twitter/scalding,scalding
1,1372,https://api.github.com/repos/twitter/finagle,finagle
2,3732,https://api.github.com/repos/ceylon/ceylon-ide...,ceylon-ide-eclipse
3,17827,https://api.github.com/repos/twitter/flockdb,flockdb
4,34552,https://api.github.com/repos/thiagoruis/dotNET...,dotNET-Grupo-2
...,...,...,...
989,131699833,https://api.github.com/repos/suimn416/2019.Spr...,2019.Spring.AI_BOSS
990,131979956,https://api.github.com/repos/capturetheworld/c...,cs151project
991,132564033,https://api.github.com/repos/idio/langdetect,langdetect
992,134135475,https://api.github.com/repos/luchizina/phpLuna,phpLuna


Time used: 1.937997817993164


,repo_id,url,name
0,7589,https://api.github.com/repos/nxtbgthng/OAuth2C...,OAuth2Client
1,8612,https://api.github.com/repos/jboss-switchyard/...,components
2,17345,https://api.github.com/repos/jboss-switchyard/...,tools
3,23282,https://api.github.com/repos/cloudera/crunch,crunch
4,24828,https://api.github.com/repos/jboss-switchyard/...,quickstarts
...,...,...,...
378,128921855,https://api.github.com/repos/huangcs3/cloudtest,cloudtest
379,129200651,https://api.github.com/repos/wer345/vscodegit,vscodegit
380,129732012,https://api.github.com/repos/MatteoCultrera/Fo...,Foody_User
381,134529452,https://api.github.com/repos/mibaldi/Architect...,ArchitectCodersG3


In the end, another analysis run is started:

In [77]:
recoveryPolyglotProjects = dbUtils.runQuery('''SELECT * FROM temp_polyglotProjects''', mute=True)
recoveryControlgroupProjects = dbUtils.runQuery('''SELECT * FROM temp_controlgroupProjects''', mute=True)
recoveryBothProjects = pandas.concat([recoveryPolyglotProjects, recoveryControlgroupProjects], axis=0)
start = time.time()
repos = list(filter(lambda tupl: repoLibrarian.isJavaRepo(tupl[0], tupl[1]), map(rowToTuple, recoveryBothProjects.iterrows())))
end = time.time()
dbUtils.log('Total Time used: '+str(end - start))

In [78]:
print(len(recoveryBothProjects))
print(len(repos))

1377
619


To simplify the code, the final module is used here:

In [80]:
repoAnalysis.runFullAnalysis(repos, repoFolder='/mnt/brick/crm20/repos/', tableName='lb_results2', logfile='recovery_log.txt')

In [81]:
dbUtils.runQuery('''
    SELECT *
    FROM crm20.lb_results2
''', mute=True)

Time used: 3.0898277759552


,sha,parent,timestamp,repo_id,additions,deletions,loc,cloc,file_count,num_methods,num_lambdas,num_comment_lines,num_reflection,num_snakes,total_indent
0,bfbb10eafcbe7ce74af7ff418f4ea2cf277b3996,fc9846d62801bc744b5dc9fc338e9532db580c0e,1597130908,613,0,347,-347,-257,-4,-36,-8,-5,0,-8,-331
1,25b81f4a68b9ec5bd694cebc92c562c4e55a0fbf,7df2c345badc749d7043293922e79e6206fd2351,1597083792,613,7,2,5,3,0,1,0,0,0,0,18
2,fc9846d62801bc744b5dc9fc338e9532db580c0e,ad1b1b38a1c5cbd811bdb7f9ee58f19b18216cd2,1597069608,613,6,3,3,1,0,0,0,0,0,0,6
3,ad1b1b38a1c5cbd811bdb7f9ee58f19b18216cd2,19b3ee2d164aa063a9a66f9fb12fc043a1c47a4f,1597064617,613,0,0,0,0,0,0,0,0,0,0,0
4,19b3ee2d164aa063a9a66f9fb12fc043a1c47a4f,3857bef92641cf923a014732810a9155d36daa7d,1597009837,613,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539272,4dab5ec84ea69ff8db6699527c67971440753c1d,983d36fd357a75e7693057f76ceeda16932d57e9,1418149334,13395535,122,1,121,87,5,13,0,0,0,8,86
539273,983d36fd357a75e7693057f76ceeda16932d57e9,65c847e0d2be48038f3483fa2718f27d5ba7740c,1417386728,13395535,0,0,0,0,0,0,0,0,0,0,0
539274,65c847e0d2be48038f3483fa2718f27d5ba7740c,f0605d5b9c3e830c35619625de69437bd67a42a3,1417030624,13395535,1,1,0,0,0,0,0,0,0,0,0
539275,f0605d5b9c3e830c35619625de69437bd67a42a3,96e54c833612e1f52b056cbbcb2f85d9e732fbc3,1416338568,13395535,0,0,0,0,0,0,0,0,0,0,0


In [66]:
display(Audio('./beep.mp3', autoplay=True))

# Third (final) iteration 

As now the technology has matured, scaling up is possible. The underlying dataset for this run is a direct superset of the two preseeding runs.

This run mainly uses the same technology as the run before, but scales the sample users up by factor 10 and decreases the factor of java to secondary code that polyglots needed. It is meant to revise and refactor the methodology and run a big stress test on it - so it can be released. 

In [99]:
tableName = 'lb_results3'

### Prepare the repos

At first, again, the repos to analyze have to be collected, downloaded, and filtered.

In [10]:
count = 0
total = len(repoTuples)
def processRepoTuple(tupl):
    isJavaRepo = False
    with io.capture_output() as output:
        global count
        count = count + 1
        print('Looking at '+str(tupl)+' - ('+str(count)+'/'+str(total)+')')
        isJavaRepo = repoLibrarian.isJavaRepo(tupl[0], tupl[1])
    dbUtils.log(output)
    return isJavaRepo

In [33]:
def prepareRepos():
    dbUtils.log('Starting to download repos')
    start = time.time()
    repos = list(filter(processRepoTuple, repoTuples))
    end = time.time()
    dbUtils.log('Total Time used: '+str(end - start))
    return repos

In [8]:
polyglotProjects = dbUtils.runQuery('''SELECT * FROM lb_polyglotProjects''', mute=True)
controlgroupProjects = dbUtils.runQuery('''SELECT * FROM lb_controlgroupProjects''', mute=True)
bothProjects = pandas.concat([polyglotProjects, controlgroupProjects], axis=0)
repoTuples = list(map(rowToTuple, bothProjects.iterrows()))

Time used: 0.5708019733428955
Time used: 0.42449331283569336


In [ ]:
repos = prepareRepos()

The log file for repo preparation, can be found under [./results/log_third_repos.txt](./results/log_third_repos.txt). It shows that approx. 16h were taken to process all repositories - that is much more than the time to actually analyze them.

Again, less than half of the repositories are available:

In [34]:
print(len(repoTuples))
print(len(repos))

16692
7676


### Dry run

In [101]:
deleteTable()
createTable()
runDeltaSuite(repos[0])
display(Audio('./beep.mp3', autoplay=True))

In [103]:
dbUtils.runQuery('''
    SELECT * FROM crm20.'''+tableName+'''
''')

Time used: 0.06114506721496582


,sha,parent,timestamp,repo_id,additions,deletions,loc,cloc,file_count,num_methods,num_lambdas,num_comment_lines,num_reflection,num_snakes,total_indent
0,bfbb10eafcbe7ce74af7ff418f4ea2cf277b3996,fc9846d62801bc744b5dc9fc338e9532db580c0e,1597130908,613,0,347,-347,-257,-4,-36,-8,-5,0,-8,-331
1,25b81f4a68b9ec5bd694cebc92c562c4e55a0fbf,7df2c345badc749d7043293922e79e6206fd2351,1597083792,613,7,2,5,3,0,1,0,0,0,0,18
2,fc9846d62801bc744b5dc9fc338e9532db580c0e,ad1b1b38a1c5cbd811bdb7f9ee58f19b18216cd2,1597069608,613,6,3,3,1,0,0,0,0,0,0,6
3,ad1b1b38a1c5cbd811bdb7f9ee58f19b18216cd2,19b3ee2d164aa063a9a66f9fb12fc043a1c47a4f,1597064617,613,0,0,0,0,0,0,0,0,0,0,0
4,19b3ee2d164aa063a9a66f9fb12fc043a1c47a4f,3857bef92641cf923a014732810a9155d36daa7d,1597009837,613,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9169,f39852a32b6cd7d4172408cbcc9bd757ae65f4d7,7074a81040c31cf8cb239a5e946dd2609761e49e,1300984314,613,146,119,27,29,0,3,0,-7,0,0,-77
9170,7074a81040c31cf8cb239a5e946dd2609761e49e,4037881e9c1da2832ad58af52f4e0b60f2441ca0,1300923433,613,117,135,-18,-20,0,-6,0,2,0,11,-33
9171,4037881e9c1da2832ad58af52f4e0b60f2441ca0,4bc9c5d53d138286134b17a730d59793f5795513,1299441062,613,132,107,25,24,0,6,1,34,-1,-19,-27
9172,4bc9c5d53d138286134b17a730d59793f5795513,8119b86c903994007f4d7fd886585e2a85c7c45b,1299324097,613,168,107,61,63,0,9,0,3,0,0,-20


## Start the run

In [ ]:
def runFullAnalysis(repos):
    createTable()
    start = time.time()
    with Pool(int(multiprocessing.cpu_count()*3/4)) as pool:
        allMetrics = pool.map(runDeltaSuite, repos)
    end = time.time()
    dbUtils.log('Total Time used: '+str(end - start))

In [ ]:
runFullAnalysis(repos)

`Total Time used: 9503.501674413681` for almost 8k repositories this is really nice.<br>
The full log file, as always, can be found under [./results/log_third_data_run.txt](./results/log_third_data_run.txt)

Teh evaluation of the results can be found in the [Results_Iteration#3](Results_Iteration#3.ipynb) notebook.

# Extracting to Module

In [36]:
%autoreload 2

To release the code for running analysis suites, the code should be extracted to the [repoAnalysis](repoAnalysis.py) and [dbUtils](dbUtils.py) modules. Especially parameters have been added to replace the global state used in the preceding runs.

This section aims at testing if the extracted methods work and to showcase their usage to potential future users.

In [52]:
repos = list(filter(lambda tupl: repoLibrarian.isJavaRepo(*tupl), [('bptlab', 'scylla', 13377331), ('bptlab', 'fcm2cpn', 4242)]))
customSuite = [repoAnalysis.loc, repoAnalysis.total_indent]
testTableName = 'lb_module_test'

In [53]:
repoAnalysis.runFullAnalysis(repos, tableName='lb_module_test', repoFolder='./local/', logfile='test_log.txt', suite=customSuite)

In [54]:
dbUtils.runQuery('''
    SELECT * FROM crm20.'''+testTableName+'''
''')

Time used: 0.02479243278503418


,sha,parent,timestamp,repo_id,additions,deletions,loc,total_indent
0,517d90b38b862f157d08a462f2cc9ed4d64e6c74,7361decd7afe0a558ba9e61e8743f23ee52dcb24,1596288222,4242,57,43,14,14
1,7361decd7afe0a558ba9e61e8743f23ee52dcb24,f37c6829849431618c5b4c011d5704ce25be8f3f,1596285127,4242,13,8,5,18
2,f37c6829849431618c5b4c011d5704ce25be8f3f,3adb5ef52051bedb9c6de7b27b21de9f0d551cc8,1596279743,4242,2,1,1,3
3,3adb5ef52051bedb9c6de7b27b21de9f0d551cc8,1773ba29816ff7dfeec8075e5a7fcf7e100d1f03,1596273974,4242,42,16,26,48
4,1773ba29816ff7dfeec8075e5a7fcf7e100d1f03,7a2e0afbfd49e8d719a684cafe23f8ef4fb2fddb,1595599925,4242,48,20,28,65
...,...,...,...,...,...,...,...,...
529,3a525dd8c0178158275815e47e6f7970372489df,a019eafb85e1d093b5a9ac75792d3c3793e0fcfe,1485388416,13377331,373,26,345,740
530,411d51771ee6c714db13c23e8912086840b84f51,9e8faf6f0b516334451c53960dee001a6e9e42c4,1485386460,13377331,237,189,47,57
531,e36dd96a1201ad35578a5e5d65a5db02135f4614,0d893a471f932b921473f72563a540da46f15fa5,1484674213,13377331,0,0,0,0
532,0d893a471f932b921473f72563a540da46f15fa5,9e8faf6f0b516334451c53960dee001a6e9e42c4,1484673978,13377331,66,22,44,135


In [55]:
dbUtils.deleteTable(testTableName)

Contents of test_log.txt:
```
========= 2020-08-16 00:59:01.439596 ==========
Time used for ('bptlab', 'fcm2cpn', 4242): 2.717569589614868

========= 2020-08-16 00:59:06.756431 ==========
Time used for ('bptlab', 'scylla', 13377331): 7.9743125438690186

========= 2020-08-16 00:59:07.256119 ==========
Total Time used: 50.08342528343201
```